#### download weight and libraries


In [0]:
import os

weight_folder = './data/'
lib_folder = './src/'
img_folder = './img/'
if not os.path.exists(weight_folder): 
  os.mkdir(weight_folder)
if not os.path.exists(img_folder): 
  os.mkdir(img_folder)
if not os.path.exists(lib_folder): 
  os.mkdir(lib_folder)

# Download weight
weight_url = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
weight_path = weight_folder + 'imagenet-vgg-verydeep-19.mat'
if not os.path.exists(weight_path):
  !curl -o $weight_path $weight_url

# Download libraries
libraries = ['vgg.py', 'constants.py', 'neural_network.py', 'utils.py']
library_url = 'https://raw.githubusercontent.com/abx67/AdvML-style-transfer/master/src/'

for lib in libraries:
  lib_path = lib_folder + lib
  lib_url = library_url + lib
  if not os.path.exists(lib_path):
    !curl -o $lib_path $lib_url
    
# Download images
style_img_name = 'van_gogh.jpg'
content_img_name = 'new_york.jpg'
img_url = 'https://raw.githubusercontent.com/abx67/AdvML-style-transfer/master/img/'

style_path = img_folder + style_img_name
style_url = img_url + style_img_name
content_path = img_folder + content_img_name
content_url = img_url + content_img_name

if not os.path.exists(style_path):
  !curl -o $style_path $style_url
if not os.path.exists(content_path):
  !curl -o $content_path $content_url

#### import modules

In [0]:
import time
import numpy as np
import PIL.Image
import sys
sys.path.append(lib_folder)
from neural_network import NeuralNetwork
from vgg import VGG

#### contents definition

In [0]:
VGG_MAT_PATH = weight_path
CONTENT_IMAGE_PATH = content_path
STYLE_IMAGE_PATH = style_path
MIXED_IMAGE_PATH = './output/'
if not os.path.exists(MIXED_IMAGE_PATH): 
  os.mkdir(MIXED_IMAGE_PATH)

CONTENT_WEIGHT = 5
STYLE_WEIGHT = 500
VARIATION_WEIGHT = 100
LEARNING_RATE = 10
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-08
MAX_ITERATION = 100
POOLING = 'max'
CHECK_POINT = 10

#### function definition

In [0]:
def load_image(file_path, max_size=None, shape=None):
    # load image and define the factor used to tranfer the image size
    image = PIL.Image.open(file_path)
    # resize by max_size
    if max_size is not None:
        factor = float(max_size) / np.max(image.size)  # image.size = [height, width, 3]
        size = np.array(image.size) * factor
        size = size.astype(int)
        image = image.resize(size, PIL.Image.LANCZOS)  # image resize with filter LANCZOS
    # resize with shape
    if shape is not None:
        image = image.resize(shape, PIL.Image.LANCZOS)
    # return image values with float data type
    return np.float32(image)


def save_image(file_path, image):
    # ensure the pixel value is int between 0 and 255
    image = np.clip(image, 0.0, 255.0).astype(np.uint8)
    # write to file
    PIL.Image.fromarray(image).save(file_path)
    return


def style_transfer(content_image_path, style_image_path, mixed_image_path,
                   content_weight, style_weight, variation_weight,
                   pooling, learning_rate, beta1, beta2, epsilon, max_iteration, check_point):
    # set the time point
    time_start = time.time()

    # load image
    content_image = load_image(content_image_path)
    style_image = load_image(style_image_path, shape=content_image.shape[:2])

    # initialize object
    vgg = VGG(VGG_MAT_PATH, pooling)
    nn = NeuralNetwork(content_image, style_image, vgg, content_weight, style_weight, variation_weight)

    # train the model
    for i, mixed_image in nn.train_model(learning_rate, beta1, beta2, epsilon, max_iteration, check_point):
        save_image(mixed_image_path + '\\v1_{}.jpeg'.format(i + 1), mixed_image)

    # print time
    time_end = time.time()
    print('Time elapsed: {} seconds'.format(round(time_end - time_start)))

    return

#### main function

In [8]:
if __name__ == '__main__':
    style_transfer(content_image_path=CONTENT_IMAGE_PATH,
                   style_image_path=STYLE_IMAGE_PATH,
                   mixed_image_path=MIXED_IMAGE_PATH,
                   content_weight=CONTENT_WEIGHT,
                   style_weight=STYLE_WEIGHT,
                   variation_weight=VARIATION_WEIGHT,
                   pooling=POOLING,
                   learning_rate=LEARNING_RATE,
                   beta1=BETA1,
                   beta2=BETA2,
                   epsilon=EPSILON,
                   max_iteration=MAX_ITERATION,
                   check_point=CHECK_POINT)

iter: 10, loss total: 7170852.5, loss content: 2416529.75, loss style: 4495433.0, loss variation: 258889.453125
iter: 20, loss total: 4637957.0, loss content: 2342397.0, loss style: 2018945.0, loss variation: 276615.1875
iter: 30, loss total: 3293741.5, loss content: 2181219.0, loss style: 872391.25, loss variation: 240131.28125
iter: 40, loss total: 2827520.25, loss content: 2023505.875, loss style: 579402.625, loss variation: 224611.765625
iter: 50, loss total: 2552133.5, loss content: 1897326.875, loss style: 434659.84375, loss variation: 220146.65625
iter: 60, loss total: 2378518.5, loss content: 1805466.0, loss style: 359070.9375, loss variation: 213981.546875
iter: 70, loss total: 2259172.0, loss content: 1739634.5, loss style: 313142.53125, loss variation: 206394.96875
iter: 80, loss total: 2174943.5, loss content: 1691433.875, loss style: 283338.09375, loss variation: 200171.40625
iter: 90, loss total: 2118692.5, loss content: 1654400.25, loss style: 268907.0, loss variation: 1